# Introduction

**Incarceration & COVID-19: How Jails Respond to COVID**<br>

This project scrapes daily jail roster information to create a large dataset. This dataset is designed to analyze how jail populations have fluctuated in response to COVID-19. Research centers on explaining why county jails in different parts of the United States have responded differently to the pandemic over time. 

A separate but related idea for this dataset analyzes the impact of pandemic-related jail population declines on local crime. This project uses daily jail roster population counts as the focal variable and the analysis uses group-based trajectory modeling. Our scraped data will address gaps in the [Vera](https://github.com/vera-institute/jail-population-data) dataset.

We start by comparing Washington and New York states because they dealt with COVID-19 at the early on-set of the pandemic. Below is a list of the data points to collect to harmonize with the Vera data.
- County Name
- State Name
- Daily Population Counts
- Reporting Jail Name

# Imports

In [215]:
# Import standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# API libraries
import re
import os
import time
import random
import requests
from os import system   
from math import floor
from copy import deepcopy

# Scraping libraries
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import json
# Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from scrapy_selenium import SeleniumRequest
import scrapy

import scrapy
from scrapy_selenium import SeleniumRequest
import sqlite3

# States

Be sure to check for APIs in addition to scraping. Will include NY, WA and FL.

## Washington

### Whitman

In [232]:
url = "http://www.whitmancountyjail.org/"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

In [276]:
listy = driver.find_elements_by_css_selector('h4')

for x in listy[:50]:
    if len(x.text) > 0:
        print(x.text)

COVID-19 Alerts
Visiting Closed
Work Release Shut-Down
ARLAND, ANDREW MICHAEL (34 years old) -- Inmate Number: 1569
CAMERON, ZANNAN ANTHONY (24 years old) -- Inmate Number: 465632
CARGILL, JOSHUA O'HARA (43 years old) -- Inmate Number: 459693
CRUZ BARRERA, JASON REY (22 years old) -- Inmate Number: 417523
DICKERSON, PATRICIA LYNN (36 years old) -- Inmate Number: 424584
GAUTHIER, JEREMY VICTOR (31 years old) -- Inmate Number: 466484
GILEAD, HEATHER EVE (21 years old) -- Inmate Number: 466531
HAYES, PHILLIP ALAN (59 years old) -- Inmate Number: 16806
HINOJOS, ALEX JOSEPH DAVID (31 years old) -- Inmate Number: 80192
JOHNSON, EVERETT CLARK (65 years old) -- Inmate Number: 19821
MARCHINI, TIMOTHY PAUL (61 years old) -- Inmate Number: 458478
MARTINEZ GARZA, ZACARIAS (24 years old) -- Inmate Number: 463422
MEAD, MARVIN HOWARD (53 years old) -- Inmate Number: 156118
NANCE, KYLE BRANDT (23 years old) -- Inmate Number: 271494
NANIK, DANIEL DOUGLAS (36 years old) -- Inmate Number: 142629
PEARSON,

In [ ]:
location = driver.find_element_by_xpath('//*[@id="form1"]/footer').text
location

In [278]:
from datetime import datetime # Current date time in local system )

In [281]:
JWhitman = (location[:19])
CWhitman = (location[:7])
SWhitman = (location[47:49])
DWhitman = datetime.now()
PWhitman = (len(listy))-10

print('Jail = ',JWhitman)
print('County = ',CWhitman)
print('State = ',SWhitman)
print('Date = ',DWhitman)
print('Population = ',PWhitman)

Jail =  Whitman County Jail
County =  Whitman
State =   W
Date =  2020-06-06 15:39:27.296722
Population =  28


### Spokane

#### Selenium

In [203]:
# url = "https://www.spokanecounty.org/352/Inmate-Roster'"
# driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
# driver.implicitly_wait(3)
# driver.get(url)

In [209]:
# print(driver.page_source)
# keep this cell, has a long print out but want the code available for future use

In [214]:
# Outter html <h1 style="margin-top:0px;">Saturday, June 6, 2020</h1>

# date = driver.find_elements_by_xpath('//*[@id="aspnetForm"]/div[3]/h1')
# len(date)


0

In [230]:
class PrisonSpider(scrapy.Spider):
    name = 'Prison'

#     def remove_characters(self, value):
#         return value.strip('\xa0')
    
    def start_requests(self):
        yield SeleniumRequest(
            url='https://www.spokanecounty.org/352/Inmate-Roster',
            wait_time=3,
            callback=self.parse
        )

    def parse(self, response):
        products = response.xpath("//*")
        for product in products:
            yield {
                'Date': product.xpath('//*[@id="aspnetForm"]/div[3]/h1').get(),
                'County': product.xpath("/html/head/title").get()\
#                 'State'
#                 'Pop_Count'
#                 'Jail'
            }

In [ ]:
driver.close()

In [ ]:
STOP

#### HTML

In [19]:
# Defining the url of the site
base_site = "https://www.spokanecounty.org/352/Inmate-Roster"

# Making a get request
response = requests.get(base_site)
response.status_code

200

In [20]:
# Extracting the HTML
html = response.content

# Checking that the reply is indeed an HTML code by inspecting the first 100 symbols
html[:100]

b'\r\n\r\n<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n\r\n\t<meta http-equiv="Content-type" content="text/html'

In [21]:
# Convert HTML to a BeautifulSoup object.
soup = BeautifulSoup(html, "html.parser")

In [22]:
# Exporting the HTML to a file
with open('Spokane_response.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

In [54]:
soup.find_all('title')

[<title>Inmate Roster | Spokane County, WA</title>,
 <title>Arrow Left</title>,
 <title>Arrow Right</title>,
 <title>Slideshow Left Arrow</title>,
 <title>Slideshow Right Arrow</title>]

In [24]:
soup.find_all('div', class_ = 'container')
# soup.find_all('li')

[]

In [52]:
mydivs = soup.findAll("div", {"class": "container"})
print(mydivs)

[]


### Okanogan

In [ ]:
# https://okanogansheriff.org/
# (under Daily Jail Inmate Log)

### Jefferson

In [ ]:
# https://co.jefferson.wa.us/174/Jail-Inmate-Search
# (To view the full inmate roster click the Clear button then the Search button.)

### Grant

In [ ]:
# https://www.grantcountywa.gov/SHERIFF/Corrections/Inmate-Roster.htm

### Gray's Harbor

In [ ]:
# http://ghlea.com/JailRosters/GHCJRoster.html

### Ferry

In [ ]:
# https://www.ferry-county.com/Courts%20and%20Law/Inmate%20Roster/Inmate_Roster_Page.html
# (in the section that says "MAY 11, 2020 - 8 inmates")

### Clallam

In [ ]:
# https://websrv23.clallam.net/NewWorld.InmateInquiry/WA0050000/

### Adams

In [ ]:
# https://www.co.adams.wa.us/government/jail_roster_and_booking_information/index.php
# (View Jail Roster Information)